Import bibliotek

In [11]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

from pathlib import Path
!pip install langchain sentence-transformers faiss-cpu pypdf transformers torch langchain-community
from langchain.document_loaders import PyPDFLoader, TextLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.vectorstores import FAISS
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoModelForSeq2SeqLM
from langchain.chains import RetrievalQA, ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
from langchain.prompts import PromptTemplate
from langchain.schema import Document
import re
import os
import torch
from langchain_community.document_loaders import PyPDFLoader

os.environ["HF_TOKEN"] = "TWÓJ_HF_TOKEN"
!pip install transformers torch langchain faiss-cpu sentence-transformers --quiet

Mounted at /content/drive


Linki do tych artykułów:

- Samoloty pasażerskie najczęściej wykorzystywane do przewozów – studium przypadku Boeninga i Airbusa : tendencje rozwoju
- Bezpieczeństwo przewozu pasażerów oraz ładunków w jednoosobowych i bezpilotowych statkach powietrznych
- Znaczenie rozwoju technologii dla konkurencyjności pasażerskiego transportu lotniczego
- ZMIANY W KONSTRUKCJACH I WYPOSAŻENIU CYWILNYCH STATKÓW PASAŻERSKICH W KONTEKŚCIE ZWIĘKSZENIA BEZPIECZEŃSTWA …

Teraz wczytanie artykułów na temat samolotów z dysku i podział na fragmenty

In [12]:
pdf_dir = "/content/drive/MyDrive/Colab Notebooks/"

file_paths = [
    os.path.join(pdf_dir, f)
    for f in os.listdir(pdf_dir)
    if f.lower().endswith(".pdf")
]

all_docs = []

for file_path in file_paths:
    file = Path(file_path)
    if file.is_file():
        try:
            loader = PyPDFLoader(file_path)
            docs = loader.load()

            print(f"Załadowano {len(docs)} stron(y) z pliku: {file.name}")

            all_docs.extend(docs)
        except Exception as e:
            print(f"Błąd podczas ładowania pliku {file.name}: {e}")
    else:
        print(f"Plik nie znaleziony: {file_path}")

text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,
    chunk_overlap=100,
    length_function=len,
    separators=["\n\n", "\n", " ", ""]
)

docs = text_splitter.split_documents(all_docs)

print(f"\nŁączna liczba stron załadowana: {len(all_docs)}")
print(f"Łączna liczba dokumentów po podziale: {len(docs)}")


Załadowano 24 stron(y) z pliku: Pre _Szwagrzyk_W poszukiwaniu_1_2015_JoTL.pdf
Załadowano 270 stron(y) z pliku: strona bezpieczenstwo przewozu www.pdf
Załadowano 26 stron(y) z pliku: Zmiany_w_konstrukcjach_i_wyposażeni.pdf
Załadowano 23 stron(y) z pliku: Znaczenie_rozwoju_technologii_dla_k.pdf

Łączna liczba stron załadowana: 343
Łączna liczba dokumentów po podziale: 1994


Stworzenie bazy wektorowej

In [21]:
embedding_model_name = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2"
embeddings = HuggingFaceEmbeddings(model_name=embedding_model_name)
index_dir = "faiss_index_working"
db = FAISS.from_documents(docs, embeddings)
db.save_local(index_dir)

Inicjalizacja modelu Q&A

In [24]:
qa_model_name = "google/flan-t5-large"
tokenizer = AutoTokenizer.from_pretrained(qa_model_name)
model = AutoModelForSeq2SeqLM.from_pretrained(qa_model_name)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)
model.eval()

custom_prompt = (
    "Answer the question based only on the following context. If you cannot find the answer in the context, say that I cannot find the answer.'\n\n"
    "Context: {context}\n\n"
    "Question: {question}\n\n"
    "Answer:"
)

K = 3
MAX_TOKENS = 512

print("Type 'exit/quit/bye/goodbye' to exit\n")

while True:
    query = input("\nWrite your question: ").strip()
    if query.lower() in ["exit", "quit", "bye", "goodbye"]:
        print("See ya!")
        break
    if not query:
        print("Question cannot be empty.")
        continue

    try:
        retrieved_docs = db.similarity_search(query, k=K)

        context_parts = []
        source_citations = []

        for i, d in enumerate(retrieved_docs):
            content = (d.page_content or "").strip()
            context_parts.append(content)

            source = d.metadata.get('source', 'Unknown')
            page = d.metadata.get('page', 'Unknown page')
            citation_info = {
                'doc_number': i + 1,
                'source_name': os.path.basename(source) if source != 'Unknown' else 'Unknown',
                'page_number': page,
                'content_preview': content[:100] + '...' if len(content) > 100 else content
            }
            source_citations.append(citation_info)

        context = "\n\n".join(context_parts)

        prompt_text = custom_prompt.format(context=context, question=query)

        inputs = tokenizer(prompt_text, return_tensors="pt", truncation=True, max_length=1024)
        inputs = {k: v.to(device) for k, v in inputs.items()}

        with torch.no_grad():
            outputs = model.generate(
                **inputs,
                max_new_tokens=MAX_TOKENS,
                do_sample=False,
                temperature=0.1
            )

        answer = tokenizer.decode(outputs[0], skip_special_tokens=True)

        print(f"AI Assistant:")
        print(f"{answer}\n")

        if retrieved_docs and source_citations:
            print(f"Sources used:")

            for citation in source_citations:
                print(f"[{citation['doc_number']}] {citation['source_name']}")
                print(f"Page: {citation['page_number']}")
                print(f"Excerpt: {citation['content_preview']}")

            print(f"\nCitation:")
            for citation in source_citations:
                if citation['source_name'] != 'Unknown':
                    print(f"   [{citation['doc_number']}] {citation['source_name']}, p. {citation['page_number']}")

        elif not retrieved_docs:
            print("No relevant documents found in the database.")

    except Exception as e:
        print(f"Error processing question: {e}")
        print("Try again with a different question.")

Type 'exit/quit/bye/goodbye' to exit


Write your question: Ile samolotów pasażerskich dostarczył Airbus w 2014 roku i jak ta liczba wypadała w porównaniu z Boeingiem?
AI Assistant:
626

Sources used:
[1] Pre _Szwagrzyk_W poszukiwaniu_1_2015_JoTL.pdf
Page: 9
Excerpt: (źródło: opracowanie własne na podstawie [7])
Obecnie w roku 2015, począwszy od 1 stycznia do dnia 3...
[2] Pre _Szwagrzyk_W poszukiwaniu_1_2015_JoTL.pdf
Page: 7
Excerpt: nie. W ostatnich latach, można zaobserwować również większy popyt na
produkty Airbusa, gdyż dostarcz...
[3] Pre _Szwagrzyk_W poszukiwaniu_1_2015_JoTL.pdf
Page: 16
Excerpt: lat 2005-2014 z udziałem samolotów pochodzących od 2 największych
producentów samolotów pasażerskich...

Citation format:
   [1] Pre _Szwagrzyk_W poszukiwaniu_1_2015_JoTL.pdf, p. 9
   [2] Pre _Szwagrzyk_W poszukiwaniu_1_2015_JoTL.pdf, p. 7
   [3] Pre _Szwagrzyk_W poszukiwaniu_1_2015_JoTL.pdf, p. 16

Write your question: Jakie są główne zalety i charakterystyka samolotów rodziny A320 fir

Tu przykład, że wskazał dobrze:

Write your question: Ile samolotów pasażerskich dostarczył Airbus w 2014 roku i jak ta liczba wypadała w porównaniu z Boeingiem?

AI Assistant:
626

Sources used:

[1] Pre _Szwagrzyk_W poszukiwaniu_1_2015_JoTL.pdf
Page: 9

Excerpt: (źródło: opracowanie własne na podstawie [7])
Obecnie w roku 2015, począwszy od 1 stycznia do dnia 3...

[2] Pre _Szwagrzyk_W poszukiwaniu_1_2015_JoTL.pdf
Page: 7

Excerpt: nie. W ostatnich latach, można zaobserwować również większy popyt na
produkty Airbusa, gdyż dostarcz...

[3] Pre _Szwagrzyk_W poszukiwaniu_1_2015_JoTL.pdf
Page: 16

Excerpt: lat 2005-2014 z udziałem samolotów pochodzących od 2 największych
producentów samolotów pasażerskich...

Citation format:

   [1] Pre _Szwagrzyk_W poszukiwaniu_1_2015_JoTL.pdf, p. 9

   [2] Pre _Szwagrzyk_W poszukiwaniu_1_2015_JoTL.pdf, p. 7

   [3] Pre _Szwagrzyk_W poszukiwaniu_1_2015_JoTL.pdf, p. 16

====================================================================

Tu ciekawe, bo wskazał w 1. źródle dobry pdf i dobrą stronę, ale nie znalazł odpowiedzi XD

Write your question: Jakie są główne zalety i charakterystyka samolotów rodziny A320 firmy Airbus?

AI Assistant:
I cannot find the answer.

Sources used:

[1] Znaczenie_rozwoju_technologii_dla_k.pdf
Page: 11

Excerpt: Źródło: opracowanie własne na podstawie Pisarek-Bartoszewska, 2020: 136.
Bardzo wyraźnie widać, że s...

[2] Pre _Szwagrzyk_W poszukiwaniu_1_2015_JoTL.pdf
Page: 5

Excerpt: Planowaną przez Airbusa odpowiedzią w wyścigu technologicznym
lotniczych gigantów jest wprowadzenie ...

[3] Znaczenie_rozwoju_technologii_dla_k.pdf
Page: 10

Excerpt: Linie lotnicze skupiają swoją flotę wokół jednego modelu lub określonej rodziny
samolotów. Taką opc...

Citation format:

   [1] Znaczenie_rozwoju_technologii_dla_k.pdf, p. 11

   [2] Pre _Szwagrzyk_W poszukiwaniu_1_2015_JoTL.pdf, p. 5

   [3] Znaczenie_rozwoju_technologii_dla_k.pdf, p. 10

======================================================

Write your question: bye

See ya!

:)

Fun fact: próbowałem użyć radlab oraz retrievala i pokazało mi, że nie jest publiczny XD

Długo się z tym męczyłem, ale się wiele nauczyłem za to :))